## Methodology

To give a run-down on what will be done here. 

- *Part one* will be a little data exploration into the trees dataset. We will look at the types of trees and how many are planted in the city. After that, we will convert that dataframe into a geopandas dataframe 

- *Part two* will be combining a statistical dataset that contains population, population density, and low income status, and a geodataset containing a similar field *DAUID* in order to georeference the statistial dataset. 

- *Part three* will aggregate the newly georeferenced dataset in order to extract the buroughs of interest and examine population density and percent low income. 

- *Part four* will examine the amount of trees present per burough

- *Part five* will be the final statistical analysis of the three variables to see if there is any coorelation. 


#### First, we need to install all of our packages, for this, just run the cell below:

In [ ]:
#run this cell

!pip install  pyogrio rioxarray earthpy matplotlib pandas geopandas missingno mapclassify folium ipyleaflet contextily


In [ ]:
#and run this cell

import matplotlib
import matplotlib.pyplot as plt
import folium
import mapclassify
import contextily
from scipy.stats import linregress
from ipyleaflet import Map, basemaps
from ipyleaflet import GeoJSON

## Part 1
### The first file we will take a look at is the directory of all the public trees in the Agglomeration of Montreal

In [ ]:
#importing the pandas module
import pandas as pd 

public_trees = pd.read_csv('arbres-publics.csv', low_memory = False)
#doing fillna to add zeros in cells that may not have a value in order to prevent any errors 
public_trees.fillna(0, inplace=True)

public_trees

#### There is a lot going on in this file, so the *csv* is also available in the notebook which can help you follow along with what exactly is going on.  

1. First of all, we have the **ARROND_NOM** which is the name of the burough that the tree is situated;

2. Second, we have **Emplacement,** which is where the tree is placed (Sidewalk, park, etc.)

3. Next two are the **X and Y coordinates** in the *Projection: Québec Modified Transverse Mercator(NAD83), Québec MTM Zone 8.*

4. After that, we have the **SIGLE**, this is an Acronym composed of the first two letters of the *genus, species* and *cultivar.*

5. The **DHP** is the is the *diameter at breast height* , this is the diameter measure of the tree, generally, the trees in my experience will range from 15cm to over 3m (usually really old maple trees), but the trees are typically planted when the diameter reaches around 10cm. 

6. The **NOM_PARC** is the name of the park if the tree is planted in a park, we will see whether or not we keep this file when we start looking more into our file. 

7. The last two columns are the **Latitude and Longitude**, both in *WGS 84*. 

#### Lets see how many types of trees exist on the public tree registry:

In [ ]:
tree_types = public_trees['SIGLE'].value_counts()

tree_types

#### It looks like there are 715 types of trees on the public registry! That is a lot of trees! 
#### Lets see when this abundance of the same tree ends by using the .head() on this variable

#### We will look at the top 10 species here:

In [ ]:
tree_types.head(10)

### we can see that the main types/cultivars of trees are:

1. Maple *(AC)*; 

2. Ash *(FR)*; 

3. Linden *(TI)*;

4. Hackberry *(CE)*;

5. Honey Locust (*GL)*;

6. Gymnocladus *(GY)* also known as the kentucky coffee tree; 

7. Blue Spruce *(PI)*; 

8. Elm *(UL)*

#### Lets see how many types of Maple trees there are. 

#### The maple trees, or *Acer,* which represents all maple tree and shrubs that exist. So this means we will be searcging for all of the variables in the **SIGLE** column that begin with **AC** for *Acer*  

In [ ]:
#extract each variable in the SIGLE column that begins with AC for ACER
maple = public_trees[public_trees['SIGLE'].str[:2] == 'AC']

maple = maple['SIGLE'].value_counts()

maple

#### There are 101 different types and cultivars of Maple trees! Lets see if we can narrow this down to some we know:

In [ ]:
maple.head(10)

#### So looking at the first 10 we have:

- ACSA - Acer Saccharrinium (silver maple)

- ACPL - Acer Platanoides (norway maple)

- ACSC - Acer Saccharum (sugar maple)

- ACFR - Acer x Freemanii (freemanii maple)
*which is the cross of a red maple and sugar maple, very very popular here*

- ACNE - Acer Negundo (box-elder or ash-leaved maple) 
*Fun fact about this tree: there is a very common confusion between the ash tree and this tree, and a lot of citizens will call in to cut down what they think is their ash tree, but find out that it is a maple tree when an inspector comes to check it out. I was that inspector for part of a summer stage**

- ACRU - Acer Rubrum (red maple)

##### Some of them have more letters in their acronyms, this isnt super important, it is just the cultivar, so sometimes it is named after the pepiniere it was grown in, or simply the color the leaves may turn at certain times of the year. 

#### Lets look at all the types of ash trees now, these are the *FR* for **Fraxinus** 

In [ ]:
#extract each variable in the SIGLE column that begins with FR for 
ash = public_trees[public_trees['SIGLE'].str[:2] == 'FR']

ash = ash['SIGLE'].value_counts()

ash.count()

#### It looks like we have 43 types of ash trees, lets look at them all since they will all actually appear when i call this variable:

In [ ]:
ash

####  The most prominent types of Ash Trees are; 

- *FRAM* is an american ash tree, or a white ash

- *FRPE* is a pennsylvanian ash tree, or a green ash

- *FRNI* is the black ash tree


#### Lets look at the next prominent tree on our list from eariler, the *TI* or **Tilia**, which is the basswood/linden tree

In [ ]:
linden = public_trees[public_trees['SIGLE'].str[:2] == 'TI']

linden = linden['SIGLE'].value_counts()

linden

#### Here we have:

- *TICO* which is tilia cordata (little leaf linden, native to europe)

- *TIAM* which is tilia americana (american basswood)

- *TIMO* which is tilia mongolica (mongolian lime, which is actually a smaller tree compared to the other two)

### Now we will convert this file to a geopandas file so that we can merge it with other spatial data

In [ ]:
import geopandas as gpd

geo_trees = gpd.GeoDataFrame(public_trees, geometry=gpd.points_from_xy(public_trees.Longitude, public_trees.Latitude))
geo_trees.rename(columns={"ARROND_NOM":"NOM"}, 
              inplace = True)
# We are changing ARROND_NOM here to match NOM in the event I decide to merge or not with other data
geo_trees

## Part 2

### The next data we will upload is census data. 

#### This data contains population, population density, and percent low income per dissemination area

In [ ]:
pop_data = pd.read_csv('MTL_DATA.csv')
 

pop_data.fillna(0, inplace=True)

pop_data

#### Unlike the other file we uploaded, this is not a geospatial dataset, so in order to add coordinates to these values, we need to upload another shapefile that contains similar data so that we can merge the data into one. 

In [ ]:
shapefile = gpd.read_file('dissemination_area.shp', engine = 'pyogrio')

#this is to turn DAUID into an integer in order to merge in the next step
to_convert = {'DAUID': 'int64'}
shapefile = shapefile.astype(to_convert)

shapefile

#### The column we will merge on is the DAUID, this will ensure that we extract only montreal data for this analysis. 

In [ ]:
pop_gdf = pd.merge(
    shapefile,
    pop_data,
    how="inner",
    on='DAUID',
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

pop_gdf

#### Now that our population data is spatially referenced, we can merge it with montreal burough data using a function we made in lab 7

In [ ]:
#first we need to upload this geojson file
muni_mtl = gpd.read_file('muni_mtl.geojson', engine = 'pyogrio')

muni_mtl

### Run the function below so that we can use it to spatially merge our two geodataframes. 

In [ ]:
def sjoin_labels(gdf1, gdf2):
    
#The first positional argument must expect a geodataframe containing polygons to which you would like to join a territorial definition.
   
    gdf1 = gdf1.to_crs('EPSG:2950')  #ensure both geodatasets are in MTM-8
    gdf2 = gdf2.to_crs('EPSG:2950') 

    #extract the centroids from the first gdf passed to it in order to then perform a spatial join.
    gdf1['centroids'] = gdf1['geometry'].centroid #derive the centroid for each DA polygon and assign it to a column called 'centroid'
    gdf1.set_geometry('centroids', inplace=True) 
    
#The second positional argument must expect a geodataframe containing polygons from which these labels will be derived.    
    #we attach the municipal boundary labels to each intersecting DA centroid:   
    gdf3 = gdf1.sjoin(gdf2)
    #reset the geometries to its original geometry once the join is complete    
    gdf3.set_geometry('geometry', inplace=True)
    
    return gdf3

### Now we can join the two datasets, do this by running the code below:

In [ ]:
montreal_data = sjoin_labels(pop_gdf, muni_mtl)
montreal_data

## Part 3

#### Now we will aggregate the data by burough so we can analyse the statistics by burough

In [ ]:
montreal_data = montreal_data.dissolve(by = "NOM", aggfunc={'POP2021':'sum', 'POPDENS':'sum', 'TOTLOWINC':'sum'})


#Next we will add a column that will give us percent low income.

montreal_data = montreal_data.assign(PERCLOWINC = ((montreal_data['TOTLOWINC']/montreal_data['POP2021'])*100))

#Since all of the cities here are not involved with the public trees dataset so lets clean this up by dropping some rows

montreal_data = montreal_data.drop(["Baie-d'Urfé", "Beaconsfield", "Côte-Saint-Luc", "Dollard-des-Ormeaux",
                                   "Dorval", "Hampstead", "Kirkland", "L'Île-Bizard-Sainte-Geneviève", "L'Île-Dorval",
                                   "Lachine", "Mont-Royal", "Montréal-Est", "Montréal-Ouest", "Outremont",
                                    "Pointe-Claire", "Sainte-Anne-de-Bellevue", "Senneville", "Westmount", "Anjou" ])

                                    
montreal_data



### Lets look at the buroughs that have the highest percentage of low income households and the buroughs that have the highest population density per square km. 

In [ ]:
#this one will reorder the columns from highest % low-income to lowest % low-income

montreal_data.sort_values(by=['PERCLOWINC'], ascending=False)

#### If we decide to look at the buroughs that have more than 20% of households at low-income status, we have:
1. Ville-Marie
2. Villeray-Saint-Michel-Parc-Extension
3. Le Plateau-Mont-Royal

#### Alternatively, if we look at the three buroughs that are included in the agglomeration of Montreal that have less than 10% of low-income households, he have:
1. Rivière-des-Prairies-Pointe-aux-Trembles
2. Pierrefonds-Roxboro
3. Verdun

In [ ]:
#this one will reorder the columns from highest population density to lowest population density

montreal_data.sort_values(by=['POPDENS'], ascending=False)

#### If we decide to look at the top three most dense buroughs we have:
1. Villeray-Saint-Michel-Parc-Extension
2. Côte-des-Neiges-Notre-Dame-de-Grâce
3. Le Plateau-Mont-Royal

#### If we look at the three least dense buroughs we have:
1. Pierrefonds-Roxboro
2. Rivière-des-Prairies-Pointe-aux-Trembles
3. Lasalle



In [ ]:
# I will try to merge both datasets now to try to make a map, first i need to make sure the 'NOM' which will be the column I am using, match in both datasets, you can see in the trees dataset that there is spacing between some of the dashes so we will fix this. 

#geo_trees['NOM'].value_counts() #this will show us all of the burough names

geo_trees['NOM'] = geo_trees['NOM'].replace({'Rosemont - La Petite-Patrie': 'Rosemont-La Petite-Patrie',
                                             'Ahuntsic - Cartierville': 'Ahuntsic-Cartierville',
                                            'Mercier - Hochelaga-Maisonneuve': 'Mercier-Hochelaga-Maisonneuve',
                                            'Côte-des-Neiges - Notre-Dame-de-Grâce': 'Côte-des-Neiges-Notre-Dame-de-Grâce',
                                            'Villeray-Saint-Michel - Parc-Extension': 'Villeray-Saint-Michel-Parc-Extension',
                                            'Rivière-des-Prairies - Pointe-aux-Trembles': 'Rivière-des-Prairies-Pointe-aux-Trembles',
                                            'Pierrefonds - Roxboro': 'Pierrefonds-Roxboro'})

geo_trees

In [ ]:
#geo_trees = geo_trees.set_crs('EPSG:2950')

## Part 4

### Examining the number of trees present in each burough

#### Now I will do some more data extraction to check out how many trees are present in the buroughs mentionned above. 

In [ ]:
Tree_Numbers_burough = geo_trees['NOM'].value_counts()

Tree_Numbers_burough

**Lowest Income:**
- Ville-Marie = 18,050
- Villeray-Saint-Michel-Parc-Extension = 25,218
- Le Plateau-Mont-Royal = 17,715

**Highest Income:**
- Rivière-des-Prairies-Pointe-aux-Trembles = 25,162 
- Pierrefonds-Roxboro = 16,454
- Verdun = 17587

**Most Dense:**
- Villeray-Saint-Michel-Parc-Extension = 25,218
- Côte-des-Neiges-Notre-Dame-de-Grâce= 26,969
- Le Plateau-Mont-Royal = 17,715

**Least Dense:**
- Pierrefonds-Roxboro = 16,454
- Rivière-des-Prairies-Pointe-aux-Trembles = 25,162
- Lasalle = 22,596

### Next we will merge our trees data with our new montreal data using NOM as our join column.

In [ ]:
montreal_dataset = pd.merge(
    montreal_data,
    geo_trees,
    how="inner",
    on=('NOM') ,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

montreal_dataset

##Part 5

### Unfortunately, my computer cannot handle making a map, or plotting anything now, so we will do some statistics to determine whether # low income, population density, and number of trees coorelate in any way
#### First i will see if income and population density depend on eachother in any way
#### Next I will see if the amount of trees depend in income or population density

In [ ]:
#First we will declare the variables so they can be easily extrated for my evaluation

PercLowInc = montreal_data['PERCLOWINC']

PopDens = montreal_data['POPDENS']

NumTrees = Tree_Numbers_burough


In [ ]:
#popdens vs. income

dens_v_inc = {'X': PercLowInc,
        'Y': PopDens}

corr = dens_v_inc['X'].corr(dens_v_inc['Y'])

corr

In [ ]:
#popdens vs. trees

dens_v_tree = {'X': PopDens,
        'Y': NumTrees}

corr1 = dens_v_tree['X'].corr(dens_v_tree['Y'])

corr1



In [ ]:
#income vs. trees

inc_v_tree = {'X': PercLowInc,
        'Y': NumTrees}

corr2 = inc_v_tree['X'].corr(inc_v_tree['Y'])

corr2

### Our results are the following:

*income v trees: -0.11*
*popdens v trees: 0.40*
*popdens v income: 0.56*

#### The type of statistical test I performed is a Pearson Coorelation. 
#### In terms of the significance; 
- correlation coefficients between -1 and -0.5 or between 0.5 and 1 indicate a strong correlation, 
- coefficients between -0.5 and -0.3 or between 0.3 and 0.5 indicate a moderate correlation, 
- coefficients between -0.3 and 0.3 indicate a weak correlation or no correlation.

#### If we use this to examine the results, we can conclude that as per the Peterson Coorelation test; 
- income and trees indicate zero coorelation
- population density and trees indicate a moderate coorelation
- income and population density indicate a strong coorelation

#### **It is important to remember that this is only one statistical test, and that coorelation does not always equal to causation**

### *In Conclusion*

#### As per the Hypothesis determined in the initial README file, not all variables coorelate, so the conclusion will be that although income and amount of public trees planted may not coorelate, there is a strong possibility that income and population density coorelate, and there is a moderate possibility that population density and amount of trees planted coorelate. 

#### In order to continue this study, it would be reccomended to examine these files on a GIS platform such as ArcPro or QGIS